# Listado de información completa
Este archivo genera un listado completo de los usuarios de todos los cursos. Cada usuario dispone de la **información demográfica** asociada a él, la información de **certificado del usuario** en el curso y la **información del curso**.

*Nota: IMPORTANTE NO DISTRIBUIR EL LISTADO GENERADO POR ESTE ARCHIVO. Para generar un listado anonimizado ver archivo MASTERFILE (Anónimo)

Funciones:
* **get_db()**: Abre los accesos para entrar a la base de datos MongoDB. Este acceso sólo es posible desde la misma instancia. Para acceder desde una ip exterior es necesario dar permisos a la nueva ip.
* **get_course()**: Obtiene sólo las colecciones relativas a los cursos
* **get_demographics()**: Obtiene de cada curso, los usuarios que están en "courseware", es decir, los que han accedido alguna vez al programa.

In [6]:
import pandas as pd
import datetime

def get_db():
    '''Access MongoDB'''
    from pymongo import MongoClient
    client = MongoClient('mongodb://???.??.?.?:?????')
    db = client.edx
    return db

def get_courses(db):
    '''Get the collections that refers to courses'''
    collections = db.collection_names(include_system_collections=True) #Obtener los nombres de todas las colecciones del MongoDB
    collections =  list(collections) #Convertir en variable tipo "list" 
    #print "Collections: \n", collections
    for item in collections: 
        if item[0:3] != "idb": #Si el comienzo es diferente de idb...
            #print item
            collections.remove(item) #Elimínala de la lista "collections"

    collections.remove("results") #For some reason it does not remove results collection.

    courses = []
    for i in collections:
        c = i.split(".")
        courses.append(c[0])

    courses = set(courses)
    #This is to remove courses that should not be analyzed by the script.
    #Includes test courses and courses never launched.
    #Should only be considered those courses that have been officially launched and closed.
    courses.remove("idb5x_1t2017")
    courses.remove("idb6x_1t2017")
    courses.remove("idb20x_1t2017")
    courses.remove("idb1x_2015_t3")
    courses.remove("idb8_1x_2t2016")
    courses.remove("idb14_1x_1t2017")
    courses.remove("idb14_2x_1t2017")
    courses.remove("idb14x_1t2017")
    courses.remove("idb9x_2015_t3")
    courses.remove("idb3x_1t2017")
    courses.remove("idb6_1x_1t2017")
    courses.remove("idb20_1x_1t2017")
    courses.remove("idb14_3x_1t2017")
    courses.remove("idb2x_2_1t2017")
    courses.remove("idb7x_2t2017")
    courses.remove("idb2x_2t2016")
    courses.remove("idb12x_1t2017")
    courses.remove("idb14x_3t2016")

    
    print "Courses: \n", courses
    return courses, collections

In [7]:
def get_demographics(db, collections):
    '''Get demographic information of the users that have enrolled in each course'''
    #Estadisticas = pd.DataFrame(columns=['Curso','Número de usuarios activos','Total de usuarios registrados','Porcentaje usuarios activos']
    
    df_all = pd.DataFrame()
    
    for index, item in enumerate(courses): #Loop for each course
        collection_demographics = item + ".demographics"
        
        if collection_demographics in collections: #Loop for each course
                
            print 'Extracting information from: ', item
            
            #-----------------DEMOGRAPHICS-----------------#
            
            #Get information from demographics
            demographics = db[item].demographics.find({}, {"id":1, "user_id":1, "gender":1, "year_of_birth":1, "level_of_education":1, "country":1})
            df =  pd.DataFrame(list(demographics)) #Transform it into dataframe
            df["course_id"] = item #Add a column with course_id
            
            #-----------------USERS-----------------#
            
            # Get personal information from collection .users
            users = db[item].users.find({"is_staff":0}, {"id":1, "username":1, "first_name":1, "last_name":1, "email":1})
            users_df = pd.DataFrame(list(users))
            
            # Merge dataframes
            df2 = pd.merge(df, users_df, left_on = "user_id", right_on = "id", how = "left")
            df2 = df2.drop('_id_x', 1)
            df2.rename(columns = {'id_x':'_id'}, inplace = True)
            # Drop columns
            df2 = df2.drop('_id_y', 1) 
            df2 = df2.drop('id_y', 1)
            
            #-----------------COURSEWARE-----------------#
            collection_courseware = item + ".courseware"
            
            if collection_courseware in collections: #If the course has no courseware...
                
                courseware = db[item].courseware.find({}, {"id":1, "student_id":1}) #Save data in a variable
                courseware_df =  pd.DataFrame(list(courseware)) #Transform it into dataframe
            
                # Number of times a user is in courseware (Progress)
                user_count = pd.DataFrame(courseware_df['student_id'].value_counts()) #Count number of times each user interacts with the platform
            
                # Prepare variable
                user_count["user_id2"] = user_count.index # Name indexes as a new column
                user_count.columns = ["Progress", "user_id"] #Name columns
            
                df3 = pd.merge(df2, user_count, on = "user_id", how = "left") #Join original dataframe with counts
            
            else:
                df3 = df2
            
            #-----------------CERTIFICATES-----------------#
            
            collection_certificates = item + ".certificates"

            if collection_certificates in collections: #If a course has the certificates collections then do, else just equal df3 to df2
            
                #Get information from collection .certificates
                certificates = db[item].certificates.find({}, {"id":1, "user_id":1, "grade":1, "status":1, "name":1, "mode":1}) #Save data in a variable
                df_certificates =  pd.DataFrame(list(certificates)) #Transform it into dataframe
                
                #Merge previously merged DataFrame with certificates
                df4 = pd.merge(df3, df_certificates, on = "user_id", how = "left") #Join current dataframe with certificates
                df4.rename(columns = {'id_x':'_id'}, inplace = True)
            
            else:
                #Just reference DataFrame used on the concatenate function below
                df4 = df3
            
            print df4[0:7]
                
            frames = [df_all, df4]
            df_all = pd.concat(frames)
            #print df_all[0:5]
    
    Date = str(datetime.datetime.now())[0:10]
    print 'The name of this file will be MASTEFILE + ', Date
    Name_csv = "MASTERFILE_" + Date + "_.csv"
      
    df_all.to_csv(Name_csv, encoding='utf-8')

In [ ]:
if __name__=="__main__":
    db = get_db() #Access the db
    courses, collections = get_courses(db) #Access the collections and bring only the courses collections
    get_demographics(db, collections)
    
    
    
    print "HECHO"